# START

In [10]:
# Libraries
import numpy as np
import pandas as pd
import json 
import jsonschema as js
import datetime
from pymongo import MongoClient
import os
import subprocess
from bson.codec_options import CodecOptions
import datetime


## functions

In [15]:
def update_valdation_schemas(db):
    # update validation schemas 
    # Compute allowed values
    allowed_specimens = db.specimens.distinct("_id")
    allowed_setups = db.exp_set_up.distinct("_id")
    if len(allowed_specimens)==0:
        allowed_specimens=['empty']
    else:
        print(f"Allowed specimens: {len(allowed_specimens)}")        
    if len(allowed_setups)==0:
            allowed_setups=['empty']
    else:
        print(f"Allowed setups: {len(allowed_setups)}")
    # Define JSON Schema validator dynamically
    new_validator ={"$jsonSchema": {"bsonType": "object",
        "required": [
        "_id",
        "measurement_no",
        'specimen_unique_id',
        'set_up_unique_id',
        'thickness',
        'diametre',
        'measurement_location',
        'measurement_timestamp',
        'polarization_angle',
        'Data'],
        "properties": {
        "_id": {"bsonType": 'objectId',"description": "must be a valid ObjectId"},
        "measurement_no": {"bsonType": "int","minimum":1 },
        "set_up_unique_id": {"bsonType": 'objectId',"enum":allowed_setups},
        "specimen_unique_id": {"bsonType": 'objectId',"enum":allowed_specimens},
        "thickness": {"bsonType": "double","minimum":0,"exclusiveMinimum":True,"description": "must be a number strictly greater than 0"},
        "diametre": {"bsonType": "double","minimum":0,"exclusiveMinimum": True,"description": "must be a number strictly greater than 0"},
        "measurement_location": {"bsonType": "string"},
        "measurement_timestamp":{"bsonType": "date","description": "must be a valid ISODate"},
        "polarization_angle":{"bsonType": "double","minimum":0,"maximum":90,"description": "must be a number in the range 0 and 90 deg"},
        "Data": {"bsonType": "array","items": {"bsonType": "array","items": {"bsonType": "double"}}},
        }
    }
    }

    # 4️⃣ Update collection validator using collMod
    result = db.command({
        "collMod": "measurements",
        "validator": new_validator,
        "validationLevel": "strict",
        "validationAction": "error"
    })

    print(" Validator updated successfully!")
    print(result)


## get server up

In [2]:
subprocess.run("cd C:\\ && mkdir data", shell=True)
subprocess.run("cd C:\\data && mkdir db", shell=True)

mongo_server="C:/mongodb-win32-x86_64-windows-8.2.1/bin/mongod.exe"
command=mongo_server+' -dbpath C:/data/db'
server=subprocess.Popen(command)


## connect to client & check database

In [ ]:
# connect to MongoDB client
client = MongoClient("mongodb://localhost:27017/")
target_db='Johan'
# --- Get existing databases ---
existing_dbs = client.list_database_names()
print("Existing databases:", existing_dbs)

# --- Check if target DB exists ---
if target_db in existing_dbs:
    print(f"✅ Database '{target_db}' already exists.")
    db = client[target_db]
    print(f"✅ I have connected to Database '{target_db}'.")
    print(f"I'm updating validation Schemas'.")
    update_valdation_schemas(db)

 
else:
    print(f"⚠️ Database '{target_db}' not found. Creating it...")

    # MongoDB creates a database automatically when you first write to it.
    db = client[target_db]
    ## Create collections 
    ## Specimens
    db.create_collection("specimens",validator={
    "$jsonSchema": {
      "bsonType": "object",
      "required": ["_id",
        "company_name",
        "company_site",
        "disassembly_day",
        "installation_date",
        "nominal_diametre",
        "nominal_thickness",
        "receipt_day",
        "specimen_id",
        "specimen_name"],
      "properties": {
        "_id": {
          "bsonType": "objectId",
          "description": "must be a valid ObjectId"
        },
        "company_name": {
          "bsonType": "string"
        },
        "company_site": {
          "bsonType": "string"
        },
        "disassembly_day": {
          "bsonType": "date",
          "description": "must be a valid ISODate"
        },
        "installation_date": {
          "bsonType": "date",
          "description": "must be a valid ISODate"
        },
        "nominal_diametre": {
          "bsonType": "double",
          "minimum":0,
          "exclusiveMinimum":True,
          "description": "must be a number strictly greater than 0"
        },
        "nominal_thickness": {
          "bsonType": "double",
          "minimum":0,
          "exclusiveMinimum":True,
          "description": "must be a number strictly greater than 0"
        },
        "receipt_day": {
          "bsonType": "date",
          "description": "must be a valid ISODate"
        },
        "specimen_id": {
          "bsonType": "string"
        },
        "specimen_name": {
          "bsonType": "string"
        }
      }
    }
  },
  validationAction= "error")
    print(f"✅ Database '{target_db}' created successfully!")
    ## Experimental/measurement set up
    db.create_collection("exp_set_up",validator={
    "$jsonSchema": {
      "bsonType": "object",
      "required": [
        "_id",
        "set_up_id",
        "set_up_name",
        "Baseband_Windowing","Baseband_FIR_filter","Baseband_DC_cancel",
        "Baseband_downsampling","Baseband_Ramps","Baseband_Samples",
        "Baseband_ADC_ClkDiv","Baseband_Averaging","System_LOG","System_Coupling",
        "System_Gain","System _AGC","System_Protocol"],
      "properties": {
        "_id": {"bsonType": "objectId","description": "must be a valid ObjectId"},
        "set_up_id": {"bsonType": "string"},
        "set_up_name": {"bsonType": "string"},
        "Baseband_Windowing":{"bsonType": "bool"},
        "Baseband_FIR_filter":{"bsonType": "bool"},
        "Baseband_DC_cancel":{"bsonType": "bool"},
        "Baseband_downsampling":{"bsonType": "int","minimum":0,"maximum":7,"exclusiveMaximum":False,"exclusiveMinimum":False,"description": "must be a number within [0,7] range"},
        "Baseband_Ramps":{"bsonType": "int","minimum":0,"maximum":7,"exclusiveMaximum":False,"exclusiveMinimum":False,"description": "must be a number within [0,7] range"},
        "Baseband_Samples":{"bsonType": "int","minimum":0,"maximum":6,"exclusiveMaximum":False,"exclusiveMinimum":False,"description": "must be a number within [0,6] range"},
        "Baseband_ADC_ClkDiv":{"bsonType": "int","minimum":0,"maximum":7,"exclusiveMaximum":False,"exclusiveMinimum":False,"description": "must be a number within [0,7] range"},
        "Baseband_Averaging":{"bsonType": "int","minimum":0,"maximum":3,"exclusiveMaximum":False,"exclusiveMinimum":False,"description": "must be a number within [0,3] range"},
        "System_LOG":{"bsonType": "bool"},
        "System_Coupling":{"bsonType": "bool"},
        "System_Gain":{"bsonType": "int","minimum":0,"maximum":5,"exclusiveMaximum":False,"exclusiveMinimum":False,"description": "must be a number within [0,5] range"},
        "System _AGC":{"bsonType": "bool"},
        "System_Protocol":{"bsonType": "int","minimum":0,"maximum":2,"exclusiveMaximum":False,"exclusiveMinimum":False,"description": "must be a number within [0,2] range"}
      }
    }
  },
  validationAction= "error")
    
    ## Create Collection for measurements
    allowed_specimens=['empty']
    allowed_setups=['empty']
    db.create_collection("measurements",validator={
    "$jsonSchema": {
      "bsonType": "object",
      "required": [
        "_id",
        "measurement_no",
        'specimen_unique_id',
        'set_up_unique_id',
        'thickness',
        'diametre',
        'measurement_location',
        'measurement_timestamp',
        'polarization_angle',
        'Data'],
      "properties": {
        "_id": {"bsonType": 'objectId',"description": "must be a valid ObjectId"},
        "measurement_no": {"bsonType": "int","minimum":1 },
        "set_up_unique_id": {"bsonType": 'objectId',"enum":allowed_setups},
        "specimen_unique_id": {"bsonType": 'objectId',"enum":allowed_specimens},
        "thickness": {"bsonType": "double","minimum":0,"exclusiveMinimum":True,"description": "must be a number strictly greater than 0"},
        "diametre": {"bsonType": "double","minimum":0,"exclusiveMinimum": True,"description": "must be a number strictly greater than 0"},
        "measurement_location": {"bsonType": "string"},
        "measurement_timestamp":{"bsonType": "date","description": "must be a valid ISODate"},
        "polarization_angle":{"bsonType": "double","minimum":0,"maximum":90,"description": "must be a number in the range 0 and 90 deg"},
        "Data": {"bsonType": "array","items": {"bsonType": "array","items": {"bsonType": "double"}}},
      }
    }
  },
  validationAction="error")

    



Existing databases: ['Johan', 'admin', 'config', 'local']
✅ Database 'Johan' already exists.
✅ I have connected to Database 'Johan'.
I'm updating validation Schemas'.
 Validator updated successfully!
{'ok': 1.0}


# Populate SPECIMEN DB

In [ ]:
# add specimen
db.specimens.insert_one({"specimen_id": "chicken 1.0", 
                         "specimen_name": "Johan",
                           "company_name": "BISE", 
                           "company_site": "Gothenburg",
                          "nominal_thickness": 0.5,
                            "nominal_diametre": 40.2,
                              "installation_date":datetime.datetime(1975, 9, 29), 
                              "disassembly_day": datetime.datetime(2025,10,12), 
                              "receipt_day": datetime.datetime(2025,10,22)})

update_valdation_schemas(db)


Allowed specimens: 2
 Validator updated successfully!
{'ok': 1.0}


# Populate SET UP DB

In [16]:
# add set up
db.exp_set_up.insert_one({
        "set_up_id": "yada yada yada 2",
        "set_up_name": "validation",
        "Baseband_Windowing":True,
        "Baseband_FIR_filter":False,
        "Baseband_DC_cancel":False,
        "Baseband_downsampling":3,
        "Baseband_Ramps":5,
        "Baseband_Samples":4,
        "Baseband_ADC_ClkDiv":6,
        "Baseband_Averaging":2,
        "System_LOG":True,
        "System_Coupling":True,
        "System_Gain":1,
        "System _AGC":False,
        "System_Protocol":1 })
update_valdation_schemas(db)

Allowed specimens: 2
Allowed setups: 2
 Validator updated successfully!
{'ok': 1.0}


# Populate MEASUREMENT DB

## filter specimen

In [36]:
specimen_id="chicken 1.0"
company_name='BISE'
company_site='Gothenburg'
############################################################################################################
############################################################################################################
df = pd.DataFrame(list(db.specimens.find({})))
df1=df[df['specimen_id']==specimen_id]
df2=df1[df1['company_name']==company_name]
df3=df2[df2['company_site']==company_site]
df3.reset_index(inplace=True)
unique_specimen_id=df3['_id'][0]


## filter set up

In [38]:
set_up_id="yada yada yada 2"
set_up_name='validation'
############################################################################################################
############################################################################################################
df = pd.DataFrame(list(db.exp_set_up.find({})))
df1=df[df['set_up_id']==set_up_id]
df2=df1[df1['set_up_name']==set_up_name]
df2.reset_index(inplace=True)
unique_set_up_id=df2['_id'][0]


## add to the db

In [44]:
# Create a 2048x2 array of random floats between 0 and 1
data = np.random.rand(2048, 2)

# Convert to list of lists (MongoDB requires JSON-serializable lists)
data_list = data.tolist()
db.measurements.insert_one({
    "specimen_unique_id": unique_specimen_id,
    "set_up_unique_id": unique_set_up_id,
    "measurement_no": 1,   
        "thickness": 0.5,
        "diametre": 24.5,
        "measurement_location": "somewhere in the middle",
        "measurement_timestamp":datetime.datetime.now(),
        "polarization_angle":45.1,
    "Data": data_list
})

InsertOneResult(ObjectId('68f8c9cf74f664caef6cd429'), acknowledged=True)

# STOP

In [45]:
server.terminate()